<a href="https://colab.research.google.com/github/RaIvPa/Improving-BERT-for-Biomedical-QA/blob/main/huggingface_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import transformers
!pip install transformers[sentencepiece]

     |████████████████████████████████| 1.1 MB 5.9 MB/s 


Initial installations and importing

In [ ]:
from transformers import pipeline

Importing pipeline object

In [ ]:
classifier = pipeline("sentiment-analysis")
classifier("This is a demo of huggingface sentiment analysis")


[{'label': 'NEGATIVE', 'score': 0.9967288970947266}]

Demoing pipelines

    feature-extraction (get the vector representation of a text)
    fill-mask
    ner (named entity recognition)
    question-answering
    sentiment-analysis
    summarization
    text-generation
    translation
    zero-shot-classification